In [ ]:
# Install simpletransformers package
!pip install simpletransformers

# Import necessary libraries
import pandas as pd
from sklearn.model_selection import train_test_split

# Load the dataset (replace with your dataset path)
data = pd.read_csv('/content/NLP_Dataset_MiniProject - customer_support_tickets_1000_unique.csv')

# Exploratory Data Analysis (EDA)
print(data.info())  # Overview of data structure
print(data['Customer Complaint'].value_counts())  # Class distribution

# Split dataset into train and validation sets
train_data, val_data = train_test_split(data, test_size=0.2, random_state=42)

# Preparing the data in the correct format for SimpleTransformers
train_df = pd.DataFrame({
    'text': train_data['Customer Complaint'],  # Corrected column name
    'labels': train_data['Category']  # Corrected column name
})

val_df = pd.DataFrame({
    'text': val_data['Customer Complaint'],  # Corrected column name
    'labels': val_data['Category']  # Corrected column name
})

print(train_df.head())  # Check the prepared training data
print(val_df.head())  # Check the prepared validation data


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 2 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Customer Complaint  1000 non-null   object
 1   Category            1000 non-null   object
dtypes: object(2)
memory usage: 15.8+ KB
None
Customer Complaint
I have been experiencing frequent disconnections with my internet service. It randomly drops multiple times a day, and I have to restart my modem each time. Despite contacting customer service, the issue persists. I work from home, and this problem is severely impacting my productivity. Please send a technician to inspect and fix this issue immediately. Case ID: 1.                                   1
I have been experiencing frequent disconnections with my internet service. It randomly drops multiple times a day, and I have to restart my modem each time. Despite contacting customer service, the issue persists. I work from home, an

# **2. Text Preprocessing**

In [ ]:
import re

# Define a function to clean text data
def clean_text(text):
    # Convert to lowercase
    text = text.lower()

    # Remove special characters and numbers
    text = re.sub(r'[^a-zA-Z\s]', '', text)

    # Remove extra whitespace
    text = text.strip()

    return text

# Apply the cleaning function to the dataset
train_df['text'] = train_df['text'].apply(clean_text)
val_df['text'] = val_df['text'].apply(clean_text)

print(train_df.head())

                                                  text                labels
29   the mobile app keeps crashing whenever i try t...       Technical Issue
535  i have been charged an extra fee on my latest ...       Billing Inquiry
695  my streaming service subscription got canceled...  Account Access Issue
557  i was supposed to receive a refund for a cance...       Billing Inquiry
836  i was supposed to receive a refund for a cance...       Billing Inquiry


## **3. Text Embedding using BERT and RoBERTa**

In [ ]:
from simpletransformers.classification import ClassificationModel

# Create a BERT model for text classification
bert_model = ClassificationModel('bert', 'bert-base-uncased', num_labels=2, use_cuda=False)  # Set use_cuda=True if using a GPU

# Create a RoBERTa model for text classification
roberta_model = ClassificationModel('roberta', 'roberta-base', num_labels=2, use_cuda=False)  # Set use_cuda=True if using a GPU

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

...

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

# **4. Model Training with BERT and RoBERTa**

In [ ]:
from sklearn.preprocessing import LabelEncoder
from simpletransformers.classification import ClassificationArgs

# Convert string labels to integer labels using LabelEncoder
label_encoder = LabelEncoder()
train_df['labels'] = label_encoder.fit_transform(train_df['labels'])
val_df['labels'] = label_encoder.transform(val_df['labels'])

# Set up model arguments with custom hyperparameters
model_args = ClassificationArgs(
    num_train_epochs=3,       # Start with 3 epochs
    train_batch_size=8,       # Use a batch size of 8
    eval_batch_size=8,        # Same for evaluation
    learning_rate=3e-5,       # Learning rate
    max_seq_length=128,       # Max sequence length
    weight_decay=0.01,        # Weight decay
    warmup_steps=0,           # Optional: adjust based on total steps
    logging_steps=50,         # Log training progress every 50 steps
    save_steps=200,           # Save the model every 200 steps
    overwrite_output_dir=True,  # Overwrite the output directory
    output_dir='outputs',     # Directory to save model outputs
)

# Train the BERT model with custom hyperparameters
bert_model = ClassificationModel(
    'bert',
    'bert-base-uncased',
    num_labels=len(label_encoder.classes_),
    args=model_args,
    use_cuda=False  # Set to True if using GPU
)
bert_model.train_model(train_df)

# Train the RoBERTa model with custom hyperparameters
roberta_model = ClassificationModel(
    'roberta',
    'roberta-base',
    num_labels=len(label_encoder.classes_),
    args=model_args,
    use_cuda=False  # Set to True if using GPU
)
roberta_model.train_model(train_df)

print("BERT and RoBERTa models trained successfully with custom hyperparameters!")

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


  0%|          | 0/1 [00:00<?, ?it/s]

Epoch:   0%|          | 0/3 [00:00<?, ?it/s]

Running Epoch 1 of 3:   0%|          | 0/100 [00:00<?, ?it/s]

Running Epoch 2 of 3:   0%|          | 0/100 [00:00<?, ?it/s]

...

Epoch:   0%|          | 0/3 [00:00<?, ?it/s]

Running Epoch 1 of 3:   0%|          | 0/100 [00:00<?, ?it/s]

Running Epoch 2 of 3:   0%|          | 0/100 [00:00<?, ?it/s]

Running Epoch 3 of 3:   0%|          | 0/100 [00:00<?, ?it/s]

BERT and RoBERTa models trained successfully with custom hyperparameters!


# **5. Evaluation on Validation Set**

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np

# Evaluate BERT on validation data
result_bert, model_outputs_bert, wrong_predictions_bert = bert_model.eval_model(val_df)

# Decode predictions back to original labels
bert_predictions = np.argmax(model_outputs_bert, axis=1)
bert_predictions_labels = label_encoder.inverse_transform(bert_predictions)
val_df['bert_predicted_label'] = bert_predictions_labels

# Print BERT evaluation results
print("BERT Evaluation Results:")
print(result_bert)

# Classification report for BERT
# Convert label_encoder.classes_ to a list of strings
target_names = [str(cls) for cls in label_encoder.classes_]
print("\nBERT Classification Report:")
print(classification_report(val_df['labels'], bert_predictions, target_names=target_names))

# Evaluate RoBERTa on validation data
result_roberta, model_outputs_roberta, wrong_predictions_roberta = roberta_model.eval_model(val_df)

# Decode predictions back to original labels
roberta_predictions = np.argmax(model_outputs_roberta, axis=1)
roberta_predictions_labels = label_encoder.inverse_transform(roberta_predictions)
val_df['roberta_predicted_label'] = roberta_predictions_labels

# Print RoBERTa evaluation results
print("\nRoBERTa Evaluation Results:")
print(result_roberta)

# Classification report for RoBERTa
# Convert label_encoder.classes_ to a list of strings
target_names = [str(cls) for cls in label_encoder.classes_]
print("\nRoBERTa Classification Report:")
print(classification_report(val_df['labels'], roberta_predictions, target_names=target_names))

0it [00:00, ?it/s]

Running Evaluation:   0%|          | 0/25 [00:00<?, ?it/s]

BERT Evaluation Results:
{'mcc': 1.0, 'eval_loss': 0.0017633295012637973}

BERT Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        48
           1       1.00      1.00      1.00        51
           2       1.00      1.00      1.00        25
           3       1.00      1.00      1.00        76

    accuracy                           1.00       200
   macro avg       1.00      1.00      1.00       200
weighted avg       1.00      1.00      1.00       200



0it [00:00, ?it/s]

Running Evaluation:   0%|          | 0/25 [00:00<?, ?it/s]


RoBERTa Evaluation Results:
{'mcc': 1.0, 'eval_loss': 0.0006317072617821395}

RoBERTa Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        48
           1       1.00      1.00      1.00        51
           2       1.00      1.00      1.00        25
           3       1.00      1.00      1.00        76

    accuracy                           1.00       200
   macro avg       1.00      1.00      1.00       200
weighted avg       1.00      1.00      1.00       200



In [ ]:
import pandas as pd

# Create a dictionary with the table data for BERT and RoBERTa
data = {
    "No.": [1, 2],
    "Model Name": ["BERT", "RoBERTa"],
    "Precision": [1, 1],  # Macro avg precision from classification reports
    "Recall": [1, 1],     # Macro avg recall from classification reports
    "F1 Score": [1, 1],   # Macro avg F1-score from classification reports
    "Accuracy": [1, 1],   # Accuracy from classification reports
    "MCC": [1, 1]       # MCC from evaluation results
}

# Convert the dictionary to a pandas DataFrame
df = pd.DataFrame(data)

# Display the table
df

,No.,Model Name,Precision,Recall,F1 Score,Accuracy,MCC
0,1,BERT,1,1,1,1,1
1,2,RoBERTa,1,1,1,1,1


# **6. Saving the Model**

In [ ]:
# Save the BERT model manually
bert_model.model.save_pretrained("bert_model")
bert_model.tokenizer.save_pretrained("bert_model")
print("BERT model saved manually!")
# Save the RoBERTa model manually
roberta_model.model.save_pretrained("roberta_model")
roberta_model.tokenizer.save_pretrained("roberta_model")
print("RoBERTa model saved manually!")

BERT model saved manually!
RoBERTa model saved manually!


# **7. Prediction on Real-World Input**

In [ ]:
# Load the saved BERT model
bert_model = ClassificationModel('bert', 'bert_model', use_cuda=False)

# Real-world input text (aligned with your dataset's context)
real_world_text = [
    "I need to cancel my order {{Order Number}}.",
    "How can I change the shipping address for my order?",
    "What is the cancellation fee for my order?"
]

# Predict the class using BERT
predictions_bert, _ = bert_model.predict(real_world_text)

# Decode predictions back to original labels
predictions_bert_labels = label_encoder.inverse_transform(predictions_bert)

# Print BERT predictions
print("BERT Predictions:")
for text, pred_label in zip(real_world_text, predictions_bert_labels):
    print(f"Text: {text} -> Predicted Intent: {pred_label}")

# Load the saved RoBERTa model
roberta_model = ClassificationModel('roberta', 'roberta_model', use_cuda=False)

# Predict the class using RoBERTa
predictions_roberta, _ = roberta_model.predict(real_world_text)

# Decode predictions back to original labels
predictions_roberta_labels = label_encoder.inverse_transform(predictions_roberta)

# Print RoBERTa predictions
print("\nRoBERTa Predictions:")
for text, pred_label in zip(real_world_text, predictions_roberta_labels):
    print(f"Text: {text} -> Predicted Intent: {pred_label}")

0it [00:00, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

BERT Predictions:
Text: I need to cancel my order {{Order Number}}. -> Predicted Intent: 0
Text: How can I change the shipping address for my order? -> Predicted Intent: 3
Text: What is the cancellation fee for my order? -> Predicted Intent: 1


0it [00:00, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]


RoBERTa Predictions:
Text: I need to cancel my order {{Order Number}}. -> Predicted Intent: 2
Text: How can I change the shipping address for my order? -> Predicted Intent: 2
Text: What is the cancellation fee for my order? -> Predicted Intent: 1
